In [3]:
import json
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import numpy as np
import pickle
from tqdm import tqdm

In [ ]:
import json
with open('./two-sigma-connect-rental-listing-inquiries/train.json') as train:
    json_data = json.load(train)
df = pd.DataFrame(json_data)

In [ ]:
# add column - NYC Zipcode 
def lat_lng(row):
    return str(row['latitude']) + ', ' + str(row['longitude'])

df['lat_lng'] = df[['latitude', 'longitude']].apply(lat_lng, axis=1)

tqdm.pandas()

geolocator = Nominatim(user_agent="rental_listing")
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.4)
df['zipcode'] = df['lat_lng'].progress_apply(geocode).apply(lambda location: location.raw.get('address', {}).get('postcode', None) if location is not None else None)

In [4]:
# add column - NYC negihborhood 
with open("two-sigma-connect-rental-listing-inquiries/nyc_zipcode.pkl", "rb") as f:
    df = pickle.load(f)
    
## New Jersey
# df.loc[df['zipcode'] == '07030-5774']
df.loc[df["zipcode"] == '07030-5774', 'zipcode'] = None
# df.loc[df['zipcode'] == '07047:07087']
df.loc[df["zipcode"] == '07047:07087', 'zipcode'] = None
# df.loc[df['zipcode'] == '07302-4522']
df.loc[df["zipcode"] == '07302-4522', 'zipcode'] = None

## Manhattan
#df.loc[df['zipcode'] == '10017']
df.loc[df["zipcode"] == '10017-6927', 'zipcode'] = '10017'
# df.loc[df['zipcode'] == '10011-6832']
df.loc[df["zipcode"] == '10011-6832', 'zipcode'] = '10011'
# df.loc[df['zipcode'] == '10037:10454']
df.loc[df["zipcode"] == '10037:10454', 'zipcode'] = '10037'
# df.loc[df['zipcode'] == '10002-1013']
df.loc[df["zipcode"] == '10002-1013', 'zipcode'] = '10002'
# df.loc[df['zipcode'] == '10009:10010']
df.loc[df["zipcode"] == '10009:10010', 'zipcode'] = '10009'
# df.loc[df['zipcode'] == '10029-6408']
df.loc[df["zipcode"] == '10029-6408', 'zipcode'] = '10029'
# df.loc[df['zipcode'] == '10025-4403']
df.loc[df["zipcode"] == '10025-4403', 'zipcode'] = '10025'
# df.loc[df['zipcode'] == '10012-3332']
df.loc[df["zipcode"] == '10012-3332', 'zipcode'] = '10012'
# df.loc[df['zipcode'] == '10018-4527']
df.loc[df["zipcode"] == '10018-4527', 'zipcode'] = '10018'

## Bronx
# df.loc[df['zipcode'] == '10454-4606']
df.loc[df["zipcode"] == '10454-4606', 'zipcode'] = '10454'

## Brooklvn
# df.loc[df['zipcode'] == '11201-1832']
df.loc[df["zipcode"] == '11201-1832', 'zipcode'] = '11201'

Bronx = ['10453', '10457', '10460', '10458', '10467', '10468', '10451', '10452', '10456', '10454', '10455', '10459', '10474',\
        '10463', '10471', '10466', '10469', '10470', '10475', '10461', '10462','10464', '10465', '10472', '10473']
Brooklvn = ['11212', '11213', '11216', '11233', '11238', '11209', '11214', '11228', '11204', '11218', '11219', '11230',\
           '11234', '11236', '11239', '11223', '11224', '11229', '11235', '11201', '11205', '11215', '11217', '11231',\
           '11203', '11210', '11225', '11226', '11207', '11208', '11211', '11222', '11220', '11232', '11206', '11221', '11237']
Manhattan = ['10026', '10027', '10030', '10037', '10039', '10001', '10011', '10018', '10019', '10020', '10036',\
            '10029', '10035', '10010', '10016', '10017', '10022', '10012', '10013', '10014', '10004', '10005', '10006',\
            '10007', '10038', '10280', '10002', '10003', '10009', '10021', '10028', '10044', '10065', '10075', '10128',\
            '10023', '10024', '10025', '10031', '10032', '10033', '10034', '10040']
Queens = ['11361', '11362', '11363', '11364', '11354', '11355', '11356', '11357', '11358', '11359', '11360', '11365',\
          '11366', '11367', '11412', '11423', '11432', '11433', '11434', '11435', '11436', '11101', '11102', '11103',\
          '11104', '11105', '11106', '11374', '11375', '11379', '11385', '11691', '11692', '11693', '11694', '11695', '11697',\
          '11004', '11005', '11411', '11413', '11422', '11426', '11427', '11428', '11429', '11414', '11415', '11416', '11417',\
          '11418', '11419', '11420', '11421', '11368', '11369', '11370', '11372', '11373', '11377', '11378']
Staten_Island = ['10302', '10303', '10310', '10306', '10307', '10308', '10309', '10312', '10301', '10304', '10305', '10314']

def neighborhood(zipcode):
    
    if zipcode in Bronx:
        return 'Bronx'
    elif zipcode in Brooklvn:
        return 'Brooklvn'
    elif zipcode in Manhattan:
        return 'Mahattan'
    elif zipcode in Queens:
        return 'Queens'
    elif zipcode in Staten_Island:
        return "staten_Island"
    else:
        return 'Not NYC'
    
df['neighborhood'] = df['zipcode'].apply(lambda row: neighborhood(row))

In [5]:
df

,index,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,zipcode,neighborhood
0,10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium,11211,Brooklvn
1,10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,low,10025,Mahattan
2,100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,high,10011,Mahattan
3,100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,low,10022,Mahattan
4,100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,low,10031,Mahattan
5,100014,2.0,4,38a913e46c94a7f46ddf19b756a9640c,2016-04-19 04:24:47,,West 18th Street,[],40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,[https://photos.renthop.com/2/6894514_9abb8592...,7995,350 West 18th Street,medium,10011,Mahattan
6,100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,[https://photos.renthop.com/2/6930771_7e3622b6...,3600,210 West 107th Street,low,10025,Mahattan
7,100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,[https://photos.renthop.com/2/6867392_b18283f6...,5645,155 West 21st Street,low,10011,Mahattan
8,100026,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,2016-04-20 02:36:35,<p><a website_redacted,Hamilton Terrace,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,[https://photos.renthop.com/2/6898799_3759be4c...,1725,63 Hamilton Terrace,medium,10031,Mahattan
9,100027,2.0,4,0,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,522 E 11th,"[Dishwasher, Hardwood Floors]",40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,[https://photos.renthop.com/2/6814332_e19a8552...,5800,522 E 11th,low,10009,Mahattan


In [6]:
# 게시된 년도, 월, 날짜, 시간 column 생성
df.created = pd.to_datetime(df.created, format='%Y-%m-%d %H:%M:%S')
df['month'] = df.created.dt.month
df['day_of_week'] = df.created.dt.weekday_name
df['hour'] = df.created.dt.hour
# 주말(1)/평일(0)  column 생성
df['weekend'] = df.apply(lambda row: 1 if row['day_of_week'] in ['Saturday', 'Sunday', 'Friday'] else 0, axis=1)

# price per bedroom
df.bedrooms[df.bedrooms == 0] = 1
df['price_per_bedroom'] = np.round(df.price / df.bedrooms, 0)

# create feature for number of photos, description length
df['num_of_photos'] = df.photos.map(len)
df['description_length'] = df.description.apply(lambda x: len(x.split(" ")))

# bedroom = 0 이면 sutdio column 추가
df['studio'] = df.bedrooms.apply(lambda x: 1 if x==0 else 0)

# 상위 10개 features value를 새로운 column으로 생성
def check(features, vals):
    for f in features:
        f = f.lower()
        if f in vals:
            return True
    return False

df['elevator'] = df.apply(lambda row: check(row['features'], ['elevator']), axis=1)
df['hardwood_floors'] = df.apply(lambda row: check(row['features'], ['hardwood floors', 'hardwood', 'hardwood floor', 'hardwood flooring' ]), axis=1)
pet_allowed = ['cats allowed', 'dogs allowed', 'pets allowed case by case', 'pets on approval', 'pet friendly', 'pets ok', 'pets', 'pets: cats/small dogs']
df['pet_allowed'] = df.apply(lambda row: check(row['features'], pet_allowed), axis=1)
doorman = ['doorman', 'full-time doorman', 'virtual doorman', 'ft doorman', '24/7 doorman', '24hr doorman', 'part-time doorman', '24/7 doorman concierge', '24-hour doorman', '24 hr doorman', '24 hour doorman', '24/7 full-time doorman concierge', ]
df['doorman'] = df.apply(lambda row: check(row['features'], doorman), axis=1)
df['dishwasher'] = df.apply(lambda row: check(row['features'], ['dishwasher', 'dish washer']), axis=1)
laundry = ['laundry in building', 'laundry in unit', 'on-site laundry', 'laundry room', 'laundry']
df['laundry'] = df.apply(lambda row: check(row['features'], laundry), axis=1)
df['no_fee'] = df.apply(lambda row: check(row['features'], ['no fee', 'reduced fee', 'low fee']), axis=1)
fitness_center = ['fitness center', 'gym/fitness', 'fitness room', 'state-of-the-art fitness center', 'fitness facility', 'fully-equipped club fitness center', 'state-of-the-art cardio and fitness club']
df['fitness_center'] = df.apply(lambda row: check(row['features'], fitness_center), axis=1)
df['pre_war'] = df.apply(lambda row: check(row['features'], ['pre-war', 'prewar', 'pre war']), axis=1)
roof_deck = ['roof deck', 'roof-deck', 'roofdeck', 'common roof deck', 'private roof deck', 'private roofdeck', 'roof deck with grills', 'roof deck with real grass', 'private roof-dec']
df['roof_deck'] = df.apply(lambda row: check(row['features'], roof_deck), axis=1)

drop = ['index', 'created', 'photos', 'building_id', 'manager_id','description', 'features','latitude', 'longitude', 'listing_id', 'street_address', 'display_address',]
df.drop(drop, axis=1, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 0 to 49351
Data columns (total 24 columns):
bathrooms             49352 non-null float64
bedrooms              49352 non-null int64
price                 49352 non-null int64
interest_level        49352 non-null object
zipcode               49224 non-null object
neighborhood          49352 non-null object
month                 49352 non-null int64
day_of_week           49352 non-null object
hour                  49352 non-null int64
weekend               49352 non-null int64
price_per_bedroom     49352 non-null float64
num_of_photos         49352 non-null int64
description_length    49352 non-null int64
studio                49352 non-null int64
elevator              49352 non-null bool
hardwood_floors       49352 non-null bool
pet_allowed           49352 non-null bool
doorman               49352 non-null bool
dishwasher            49352 non-null bool
laundry               49352 non-null bool
no_fee                49352 n